In [48]:
using Pkg
for p in ["Knet", "Plots", "IterTools", "PyCall", "ArgParse"]
    if !haskey(Pkg.installed(),p)
        Pkg.add(p);
    end
end
 
using Knet
using DelimitedFiles
using Knet: KnetArray, accuracy, progress, minibatch, cycle, adam, xavier_uniform, progress!
using Plots
using ArgParse
using IterTools: ncycle, takenth, take
using Base.Iterators: flatten
using LinearAlgebra

include("utils.jl")
include("models.jl")

function train_with_early_stopping(model, data, epochs, lr, window_size)
    early_stop_counter = 0
    prev_val_loss = 0
    iter = 0

    trnloss = []
    valloss = []
    flag = true

    function task()

        append!(trnloss, model(data))
        v_loss = val_loss(model, data)
        append!(valloss, v_loss)

        if v_loss >= prev_val_loss
            early_stop_counter = early_stop_counter + 1
        else
            early_stop_counter = 0
        end
        if early_stop_counter == window_size
            flag = false
        end

        iter = iter + 1
        prev_val_loss = v_loss

        return flag
    end

    training = adam(model, ncycle(data, epochs), lr=lr)
    progress!(flag = task() for x in (x for (i,x) in enumerate(training)) if flag)

    return 1:iter, trnloss, valloss
end

# TODO: take user inputs
struct args
    epochs
    lr
    weight_decay
    hidden
    pdrop
    window_size
end

arguments = args(200, 0.01, 5e-4, 16, 0.5, 10)

function val_loss(model,x,y)
    output = model(x)[:, idx_val]
    nll(output, y[idx_val]) + (arguments.weight_decay * sum(model.layer1.w .* model.layer1.w))
end
function val_loss(model, d)
    mean(val_loss(model,x,y) for (x,y) in d)
end

function test_loss(model,x,y)
    output = model(x)[:, idx_test]
    nll(output, y[idx_test]) + (arguments.weight_decay * sum(model.layer1.w .* model.layer1.w))
end
function test_loss(model,d)
    mean(test_loss(model,x,y) for (x,y) in d)
end

(g::GCN)(x,y) = nll(g(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(g.layer1.w .* g.layer1.w))
(m::MLP)(x,y) = nll(m(x)[:, idx_train], y[idx_train]) + (arguments.weight_decay * sum(m.layer1.w .* m.layer1.w))
(m::GCN2)(x,y) = nll(m(x)[:, idx_train], y[idx_train]) 

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


In [53]:
include("utils.jl")

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0


load_dataset (generic function with 2 methods)

In [50]:
atype = gpu() >= 0 ? KnetArray{Float32,2} : SparseMatrixCSC{Float32,Int64}

SparseMatrixCSC{Float32,Int64}

In [51]:
(m::GCN2)(x,y) = nll(m(x)[:, idx_train], y[idx_train]) 

In [54]:
adj, features, labels, idx_train, idx_val, idx_test = load_dataset("cora",3)

1
2
3
4
